In [18]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pandas as pd
from datetime import datetime
import os

# Configuration
video_source = "/kaggle/input/videos/Video/vid3.mp4"
csv_file = 'person_detections.csv'
custom_object = {'mae': tf.keras.losses.MeanAbsoluteError()}
MAX_AGE = 116.0  # Maximum age in UTK face dataset
gender_dict = {0: "Male", 1: "Female"}

In [19]:
try:
    print("Loading models.")
    face_detector = load_model('/kaggle/input/model-task2/keras/default/1/facedetectortask2.h5',
                              custom_objects=custom_object)
    age_model = load_model('/kaggle/input/model-task2/keras/default/1/agetask2.h5',
                           custom_objects=custom_object)
    gender_model = load_model('/kaggle/input/model-task2/keras/default/1/gendertask2.h5')
    print("Models loaded successfully.")
except Exception as e:
    print(f"Error loading models: {e}")
    exit(1)

Loading models.
Models loaded successfully.


In [20]:
def preprocess_image(img, size=(128, 128)):
    if not isinstance(img, np.ndarray) or img.size == 0 or len(img.shape) < 2:
        raise ValueError(f"Invalid input image: shape {img.shape if isinstance(img, np.ndarray) else 'None'}")
    
    if len(img.shape) == 2 or (len(img.shape) == 3 and img.shape[2] == 1):
        if len(img.shape) == 3:
            img = img[:, :, 0]
    else:
        try:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        except Exception as e:
            raise ValueError(f"Failed to convert to grayscale: {e}. Shape: {img.shape}")
    
    # Resizing, normalizing, and adding dimensions
    img = cv2.resize(img, size, interpolation=cv2.INTER_LANCZOS4)
    img = img.astype(np.float32) / 255.0
    img = np.expand_dims(img, axis=-1)  # Shape: (128, 128, 1)
    img = np.expand_dims(img, axis=0)   # Shape: (1, 128, 128, 1)
    return img

In [21]:
def detect_faces(frame, min_confidence=0.7, window_size=(128, 128), stride=32):
    if not isinstance(frame, np.ndarray) or frame.size == 0 or len(frame.shape) != 3 or frame.shape[2] != 3:
        print(f"Invalid frame: expected 3-channel BGR image, got shape {frame.shape if isinstance(frame, np.ndarray) else 'None'}")
        return []
    
    frame = cv2.resize(frame, (640, 480))
    height, width = frame.shape[:2]
    confirmed_faces = []
    
    for y in range(0, height - window_size[1] + 1, stride):
        for x in range(0, width - window_size[0] + 1, stride):
            window = frame[y:y+window_size[1], x:x+window_size[0]]
            if window.size == 0 or window.shape[0] != window_size[1] or window.shape[1] != window_size[0] or window.shape[2] != 3:
                print(f"Invalid window at ({x}, {y}): shape {window.shape}")
                continue
            try:
                face_input = preprocess_image(window)
                if face_input.shape != (1, 128, 128, 1):
                    print(f"Invalid face_input shape at ({x}, {y}): got {face_input.shape}")
                    continue
                confidence = face_detector.predict(face_input, verbose=0)[0][0]
                if confidence > min_confidence:
                    confirmed_faces.append((x, y, window_size[0], window_size[1]))
            except Exception as e:
                print(f"Error processing window at ({x}, {y}): {e}. Window shape: {window.shape}")
                continue
    
    return confirmed_faces

In [22]:
def predict_age_gender(face_img):
    try:
        input_img = preprocess_image(face_img)
        gender_pred = gender_model.predict(input_img, verbose=0)[0][0]
        age_pred = age_model.predict(input_img, verbose=0)[0][0]
        gender = 1 if gender_pred > 0.5 else 0
        age = float(age_pred) * MAX_AGE
        return gender, age
    except Exception as e:
        print(f"Error in age/gender prediction: {e}. Input shape: {face_img.shape}")
        raise

In [29]:
def process_frame(frame, min_confidence=0.7):
    if not isinstance(frame, np.ndarray) or frame.size == 0 or len(frame.shape) != 3 or frame.shape[2] != 3:
        raise ValueError(f"Invalid frame: expected 3-channel BGR image, got shape {frame.shape if isinstance(frame, np.ndarray) else 'None'}")
    
    frame = cv2.resize(frame, (640, 480))
    faces = detect_faces(frame, min_confidence)
    detections = []
    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]
        if face_img.size == 0 or face_img.shape[0] == 0 or face_img.shape[1] == 0:
            print(f"Invalid face image at ({x}, {y}, {w}, {h}): shape {face_img.shape}")
            continue
        gender, age = predict_age_gender(face_img)
        if(age>60):
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            detections.append({
                'Timestamp': timestamp,
                'Age': round(max(0, age)),
                'Gender': gender_dict[gender]
            })
    if detections:
        pd.DataFrame(detections).to_csv(csv_file, mode='a', header=False, index=False)
    return frame

In [28]:
try:
    if not os.path.exists(csv_file):
        pd.DataFrame(columns=['Timestamp', 'Age', 'Gender']).to_csv(csv_file, index=False)
    
    cap = cv2.VideoCapture(video_source)
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_source}")
        exit(1)
    print("Starting video pre-processing")
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of video or error reading frame.")
            break
        try:
            if frame_count % 10 == 0:
                frame = process_frame(frame)
            frame_count += 1
            if frame_count % 100 == 0:
                print(f"Processed {frame_count} frames")
        except Exception as e:
            print(f"Error processing frame {frame_count}: {e}")
            continue
    
    print(f"Processed {frame_count} frames. CSV output saved to {csv_file}")
    cap.release()
except Exception as e:
    print(f"Error in main loop: {e}")
finally:
    if 'cap' in locals():
        cap.release()

Starting video pre-processing
Processed 100 frames
Processed 200 frames
End of video or error reading frame.
Processed 202 frames. CSV output saved to person_detections.csv
